In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os
import numpy as np
import pandas as pd
import sys
import cv2


In [2]:
class FaceImage(Dataset):
    def __init__(self,label,picture_file,path,Train=True):
        self.data_path = path
        self.label = label
        self.train = Train
        self.picture_file = picture_file
    def __getitem__(self,index):

        file_path = os.path.join(self.data_path,self.picture_file[index])
        img = cv2.imread(file_path,cv2.IMREAD_GRAYSCALE)
        img = np.expand_dims(img,0)
        img =torch.tensor(img).float()
        if self.train==True:
            return torch.FloatTensor(img),self.label[index,1]
        else:
            return img

    def __len__(self):
        return len(os.listdir(self.data_path))


In [22]:
class ImageNet(nn.Module):
    def __init__(self):
        super(ImageNet,self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size =(5,5),stride =(1,1),padding = (3,3))
        self.conv2 = nn.Conv2d(6, 16, kernel_size =(4,4),stride =(1,1),padding = (2,2))
        self.conv3 = nn.Conv2d(16, 32, kernel_size =(3,3),stride =(1,1),padding = (1,1))
        self.fcn1 = nn.Linear(in_features =1152 ,out_features =512 ,bias = True)
        self.fcn2 = nn.Linear(in_features = 512,out_features =256 ,bias = True)
        self.fcn3 = nn.Linear(in_features =256 ,out_features =7 ,bias = True)

    def forward(self , x):
        x =F.relu(F.max_pool2d(self.conv1(x),2))
        #print('Tensor size and type after conv1:', x.shape, x.dtype)
        x =F.relu(F.max_pool2d(self.conv2(x),2))
        #print('Tensor size and type after conv2:', x.shape, x.dtype)
        x =F.relu(F.max_pool2d(self.conv3(x),2))
        #print('Tensor size and type after conv3:', x.shape, x.dtype)
        x = x.view(x.size(0),-1)
        #print("size:",x.shape)
        x = F.relu(self.fcn1(x))
        x = F.relu(self.fcn2(x))
        #out = torch.sigmoid(self.fcn3(x))
        out = F.softmax(self.fcn3(x),dim=1)
        
        return out

In [23]:
train_label = pd.read_csv('./train.csv')
train_label = np.array(train_label,dtype = int)
datalen  = len(train_label)
picture_file = sorted(os.listdir('./train_img'))
train_dataset = FaceImage(train_label,picture_file,'./train_img')
picture_file = sorted(os.listdir('./test_img'))
test_dataset = FaceImage(train_label,picture_file,'./test_img',Train = False)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 256)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size = 256)


In [ ]:
model = ImageNet()
print(model)
for epoch in range(100):
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    model.train()
    train_loss = 0
    correct = 0

    for batch_index ,(data, label) in enumerate(train_loader):
        
        optimizer.zero_grad()
        output = model(data)
        #print(output.shape,label.shape)
        loss = F.cross_entropy(output, label)
        #train_loss += criterion(output, label).item()
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_index * len(data), len(train_loader.dataset),100. * batch_index / len(train_loader), loss.item()),end = "\r")



ImageNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(3, 3))
  (conv2): Conv2d(6, 16, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fcn1): Linear(in_features=1152, out_features=512, bias=True)
  (fcn2): Linear(in_features=512, out_features=256, bias=True)
  (fcn3): Linear(in_features=256, out_features=7, bias=True)
)


In [ ]:
torch.save(model,"Imagenet.pkl")

In [ ]:
ans = []
for idx,test in enumerate(test_loader):
    ans.append(model(test))


In [ ]:
ans_final =[]
for a in ans:
    for pre in a:
        ans_final.append(pre)
print((ans_final[0]))
    

In [20]:
with open("cnn.csv","w") as f:
    print("id,label", file = f)
    for id,label in enumerate(ans_final):
        print("{},{}".format(id+1,label) ,file = f)







# Pytorch 不用one-hot-encoding 額外寫會出錯
